# Land Use/Land Cover Download and Reclassification

## Metadata
- **Script Name**: Download and reclassification of LULC.ipynb
- **Project**: EarthCul - Cultural Ecosystem Services Analysis
- **Author**: [Author Name]
- **Date Created**: [Creation Date]
- **Last Modified**: [Last Modification Date]
- **Version**: 1.0

## Description
This notebook downloads and processes CORINE Land Cover (CLC) data for Spanish National Parks using Google Earth Engine (GEE). The script performs automated reclassification of land use codes to standardize categories across all study areas.

## Objectives
1. Download CORINE Land Cover 2018 data for each national park
2. Apply systematic reclassification of land use categories
3. Generate standardized LULC layers for landscape metrics calculation
4. Export processed layers in GeoTIFF format with consistent spatial properties

## Study Areas
The script processes data for 8 Spanish National Parks:
- Aigüestortes i Estany de Sant Maurici (Aiguestortes)
- Ordesa y Monte Perdido (Ordesa)  
- Peneda-Gerês (Peneda)
- Sierra de Guadarrama (Guadarrama)
- Picos de Europa (Picos)
- Sierra de las Nieves (SierraNieves)
- Sierra Nevada (SierraNevada)
- Teide (Teide)

## Data Sources
- **Primary Dataset**: COPERNICUS/CORINE/V20/100m/2018 (via Google Earth Engine)
- **Spatial Resolution**: 100 meters
- **Temporal Coverage**: 2018
- **Coordinate System**: EPSG:32630 (UTM Zone 30N)
- **Study Area Boundaries**: Custom feature collection "users/cnav/EarthCul/BBs_EarthCul"

## Methodology
1. **Area of Interest Definition**: 
   - Load park boundaries from GEE asset
   - Apply 10km buffer around each park
   
2. **Land Cover Reclassification**:
   - Original CORINE classes are remapped to simplified categories
   - Artificial surfaces (codes 111-142) → consolidated code 100
   - Natural and semi-natural areas maintain original classification
   - Reclassification scheme follows [LULC_CORINE_Codes.csv](https://github.com/kampax/EarthCul/blob/main/Landscape%20variables/LULC_CORINE_Codes.csv)

3. **Data Export**:
   - Clipped to buffered park boundaries
   - Exported as GeoTIFF at 100m resolution
   - Both reclassified and original versions available

## Technical Requirements
- **Python Libraries**: `ee`, `geemap`, `os`
- **Google Earth Engine**: Authenticated account required
- **Output Format**: GeoTIFF (.tif)
- **Memory Requirements**: Variable depending on park size

## Outputs
- `LULC.tif`: Reclassified land use/land cover layer
- `Original_LULC.tif`: Original CORINE classification (optional)

## Usage Notes
- Modify the `park` variable to process different national parks
- Ensure sufficient disk space for output files
- Check GEE usage quotas for large area exports
- Verify coordinate system compatibility with downstream analysis

## References
- CORINE Land Cover: https://land.copernicus.eu/en/products/corine-land-cover
- Google Earth Engine: https://earthengine.google.com/
- Geemap Documentation: https://geemap.org/

# Land Use/Land Cover Download and Reclassification

## Metadata
- **Script Name**: Download and reclassification of LULC.ipynb
- **Project**: EarthCul - Cultural Ecosystem Services Analysis
- **Author**: Carlos Javier Navarro
- **Date Created**: 2024
- **Version**: 1.0

## Description
This notebook downloads and processes CORINE Land Cover (CLC) data for Spanish and Portuguese National Parks using Google Earth Engine (GEE). The script performs automated reclassification of land use codes to standardize categories across all study areas based on the LULC_CORINE_Codes.csv file

## Objectives
1. Download CORINE Land Cover 2018 data for each national park
2. Apply systematic reclassification of land use categories
3. Generate standardized LULC layers for landscape metrics calculation
4. Export processed layers in GeoTIFF format with consistent spatial properties

## Study Areas
The script processes data for 8 Spanish National Parks:
- Aigüestortes i Estany de Sant Maurici (Aiguestortes)
- Ordesa y Monte Perdido (Ordesa)  
- Peneda-Gerês (Peneda)
- Sierra de Guadarrama (Guadarrama)
- Picos de Europa (Picos)
- Sierra de las Nieves (SierraNieves)
- Sierra Nevada (SierraNevada)
- Teide (Teide)

## Data Sources
- **Primary Dataset**: COPERNICUS/CORINE/V20/100m/2018 (via Google Earth Engine)
- **Spatial Resolution**: 100 meters
- **Temporal Coverage**: 2018
- **Coordinate System**: EPSG:32630 (UTM Zone 30N)
- **Study Area Boundaries**: Custom feature collection "users/cnav/EarthCul/BBs_EarthCul"

## Methodology
1. **Area of Interest Definition**: 
   - Load park boundaries from GEE asset
   - Apply 10km buffer around each park, to avoid the edge effect on some landscape metrics
   
2. **Land Cover Reclassification**:
   - Original CORINE classes are remapped to simplified categories
   - Artificial surfaces (codes 111-142) → consolidated code 100
   - Natural and semi-natural areas maintain original classification
   - Reclassification scheme follows [LULC_CORINE_Codes.csv](https://github.com/kampax/EarthCul/blob/main/Landscape%20variables/LULC_CORINE_Codes.csv)

3. **Data Export**:
   - Clipped to buffered park boundaries
   - Exported as GeoTIFF at 100m resolution
   - Both reclassified and original versions available

## Technical Requirements
- **Python Libraries**: `ee`, `geemap`, `os`
- **Google Earth Engine**: Authenticated account required
- **Output Format**: GeoTIFF (.tif)
- **Memory Requirements**: Variable depending on park size

## Outputs
- `LULC.tif`: Reclassified land use/land cover layer
- `Original_LULC.tif`: Original CORINE classification (optional)

## Usage Notes
- Modify the `park` variable to process different national parks
- Ensure sufficient disk space for output files
- Check GEE usage quotas for large area exports
- Verify coordinate system compatibility with downstream analysis

## References
- CORINE Land Cover: https://land.copernicus.eu/en/products/corine-land-cover
- Google Earth Engine: https://earthengine.google.com/
- Geemap Documentation: https://geemap.org/

In [1]:
import ee
import geemap

# Initialize and authenticate Earth Engine
#Remove comment if authentication is required in Google Earth Engine
# ee.Authenticate()
ee.Initialize()

In [2]:
# Load the Features collection
BBs = ee.FeatureCollection("users/cnav/EarthCul/BBs_EarthCul")
#// Aiguestortes
#// Ordesa
#// Peneda
#// Guadarrama
#// Picos
#// SierraNieves
#// SierraNevada
#// Teide

#Filter collection by park
park = "Aiguestortes"
AOI = BBs.filter(ee.Filter.eq("PN", park))
AOI = AOI.geometry().buffer(10000)###Include a 10 km buffer around my area of interest

# Load the Corine Land Cover land use layer
clc = ee.Image("COPERNICUS/CORINE/V20/100m/2018")

# Recategorize CLC values
oldValues = [111,112,121,122,123,124,131,132,133,142,
141,
211,212,213,221,222,223,231,241,242,243,244,
311,312,313,321,322,323,324,331,332,333,334,335,
411,412,421,422,423,511,512,521,522,523
             ]

newValues = [100,100,100,100,100,100,100,100,100,100, #Artificial surfaces 
141,
211,212,213,221,222,223,231,241,242,243,244,
311,312,313,321,322,323,324,331,332,333,334,335,
411,412,421,422,423,511,512,521,522,523
             ]

# Remap raster values
clc_recat = clc.remap(oldValues, newValues, None).rename('landcover')

# Agregar la capa recategorizada al mapa
palette = [
'#ffa6ff', '#ffffa8', '#ffff00', '#e6e600', '#e68000', '#f2a64d', '#e6a600', '#e6e64d', 
'#ffe6a6', '#ffe64d', '#e6cc4d', '#f2cca6', '#80ff00', '#00a600', '#4dff00', '#ccf24d', 
'#a6ff80', '#a6e64d', '#a6f200', '#e6e6e6', '#cccccc', '#ccffcc', '#000000', '#a6e6cc', 
'#a6a6ff', '#4d4dff', '#ccccff', '#e6e6ff', '#a6a6e6', '#00ccf2', '#80f2e6', '#00ffa6', 
'#a6ffe6', '#e6f2ff'

]

# Add the recategorized layer to the map
Map = geemap.Map(center=[40, -100], zoom=4)
Map.addLayer(clc_recat, {'min': 100, 'max': 523, 'palette': palette}, "LandCover recategorized")
Map.addLayer(clc, {}, "Original LandCover")

Map.centerObject(AOI, 10)

Map

Map(center=[42.57826728642684, 0.9164188095819685], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:

import os
# Code to download the generated land use layer
## Define the output folder
base_path = os.getcwd()
out_dir = os.path.join(base_path,"../LULC/"+park)
filename = os.path.join(out_dir, 'LULC.tif')

# Clip layer
image = clc_recat.clip(AOI).unmask()

# Export LULC layer
geemap.ee_export_image(
    image, filename=filename, scale=100, region=AOI, file_per_band=False, crs= "EPSG:32630"
)


In [ ]:
# If also want to download the original land use layer

filename2 = os.path.join(out_dir, 'Original_LULC.tif')

# Clip layer
image2 = clc.clip(AOI).unmask()

#Export original layer
geemap.ee_export_image(
    image2, filename=filename2, scale=100, region=AOI, file_per_band=False
)

In [ ]:
# If you want like to download to the drive

# geemap.ee_export_image_to_drive(
#     image, description='LULC', folder='export', region=AOI, scale=100
# )